In [1]:
try:
    get_ipython().run_line_magic("reset", "-f")
    p = display
except NameError:
    p = print

import warnings

warnings.simplefilter("ignore")

import pandas as pd

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

input_dir = "march-machine-learning-mania-2025"

In [2]:
df_names = []
df_names_compact = []
df_names_detailed = []

for gender in ["M", "W"]:
    for part in ["RegularSeason", "NCAATourney", "SecondaryTourney"]:
        for result_type in ["Compact", "Detailed"]:
            if part == "SecondaryTourney" and result_type == "Detailed":
                continue
            df_name = f"{gender}{part}{result_type}Results"
            df_names.append(df_name)
            if result_type == "Compact":
                df_names_compact.append(df_name)
            else:
                df_names_detailed.append(df_name)
            print(df_name)
            path = f"../input/{input_dir}/{df_name}.csv"
            df = pd.read_csv(path)
            df["Part"] = part
            globals()[df_name] = df

season_min = WRegularSeasonDetailedResults["Season"].min()

for df_name in df_names:
    df = globals()[df_name]
    globals()[df_name] = df[df["Season"] >= season_min]

MRegularSeasonCompactResults
MRegularSeasonDetailedResults
MNCAATourneyCompactResults
MNCAATourneyDetailedResults
MSecondaryTourneyCompactResults
WRegularSeasonCompactResults
WRegularSeasonDetailedResults
WNCAATourneyCompactResults
WNCAATourneyDetailedResults
WSecondaryTourneyCompactResults


In [19]:
game_stats_1_2 = pd.DataFrame()

for df_name in df_names_detailed:
    game_stats_1_2 = pd.concat([game_stats_1_2, globals()[df_name]])

game_stats_1_2 = game_stats_1_2.reset_index(drop=True)
mask = game_stats_1_2["WTeamID"] < game_stats_1_2["LTeamID"]


def to_1_2(col_W_L):
    game_stats_1_2.loc[mask, f"{col_W_L}_1"] = game_stats_1_2[f"W{col_W_L}"]
    game_stats_1_2.loc[mask, f"{col_W_L}_2"] = game_stats_1_2[f"L{col_W_L}"]
    game_stats_1_2.loc[~mask, f"{col_W_L}_1"] = game_stats_1_2[f"L{col_W_L}"]
    game_stats_1_2.loc[~mask, f"{col_W_L}_2"] = game_stats_1_2[f"W{col_W_L}"]
    game_stats_1_2[f"{col_W_L}_1"] = game_stats_1_2[f"{col_W_L}_1"].astype(
        game_stats_1_2[f"W{col_W_L}"].dtype
    )
    game_stats_1_2[f"{col_W_L}_2"] = game_stats_1_2[f"{col_W_L}_2"].astype(
        game_stats_1_2[f"W{col_W_L}"].dtype
    )


game_stats_1_2["WLoc"] = game_stats_1_2["WLoc"].astype("category")
game_stats_1_2["LLoc"] = game_stats_1_2["WLoc"].cat.rename_categories(
    {"A": "H", "H": "A"}
)

for c in game_stats_1_2.columns:
    if c.startswith("W"):
        to_1_2(c[1:])

game_stats_1_2["Spread_1"] = game_stats_1_2["Score_1"] - game_stats_1_2["Score_2"]
game_stats_1_2["Spread_2"] = game_stats_1_2["Score_2"] - game_stats_1_2["Score_1"]
game_stats_1_2 = game_stats_1_2[[c for c in game_stats_1_2 if c[0] not in ("W", "L")]]
game_stats_1_2 = game_stats_1_2.drop(columns="NumOT")
game_stats_1_2.insert(2, "TeamID_1", game_stats_1_2.pop("TeamID_1"))
game_stats_1_2.insert(3, "TeamID_2", game_stats_1_2.pop("TeamID_2"))
game_stats_1_2 = game_stats_1_2.sort_values(
    ["Season", "DayNum", "TeamID_1", "TeamID_2"]
)
game_stats_1_2 = game_stats_1_2.reset_index(drop=True)
print(f"game_stats_1_2     {game_stats_1_2.shape[0]:,}")
p(game_stats_1_2)
print()

Y = (
    game_stats_1_2.groupby(["Season", "TeamID_1", "TeamID_2"])["Spread_1"]
    .mean()
    .reset_index()
)
print(f"Y                  {Y.shape[0]:,}")
p(Y)
print()

reg_season = game_stats_1_2[game_stats_1_2["Part"] == "RegularSeason"]
print(f"reg_season         {reg_season.shape[0]:,}")

game_stats_1_o = reg_season.rename(
    columns={c: f"{c[:-2]}_o" for c in reg_season if c[-2:] == "_1"}
)
game_stats_1_o = game_stats_1_o.rename(
    columns={c: f"{c[:-2]}_d" for c in game_stats_1_o if c[-2:] == "_2"}
)
print(f"game_stats_1_o     {game_stats_1_o.shape[0]:,}")

game_stats_2_o = reg_season.rename(
    columns={c: f"{c[:-2]}_o" for c in reg_season if c[-2:] == "_2"}
)
game_stats_2_o = game_stats_2_o.rename(
    columns={c: f"{c[:-2]}_d" for c in game_stats_2_o if c[-2:] == "_1"}
)
print(f"game_stats_2_o     {game_stats_2_o.shape[0]:,}")
print(f"game_stats_1_o+2_o {game_stats_1_o.shape[0]+game_stats_2_o.shape[0]:,}\n")

game_stats_o_d = pd.concat([game_stats_1_o, game_stats_1_o])
game_stats_o_d = game_stats_o_d.drop(columns="Part")
game_stats_o_d.insert(2, "TeamID_o", game_stats_o_d.pop("TeamID_o"))
game_stats_o_d.insert(3, "TeamID_d", game_stats_o_d.pop("TeamID_d"))
game_stats_o_d = game_stats_o_d.sort_values(
    ["Season", "DayNum", "TeamID_o", "TeamID_d"]
)
game_stats_o_d = game_stats_o_d.reset_index(drop=True)
print(f"game_stats_o_d     {game_stats_o_d.shape[0]:,}")
p(game_stats_o_d)
print()

season_stats = game_stats_o_d.rename(columns={"TeamID_o": "TeamID"})
season_stats_g = season_stats.groupby(["Season", "TeamID"])
season_stats = season_stats_g[
    [c for c in season_stats if c.endswith("_o")]
    + [c for c in season_stats if c.endswith("_d")]
]
season_stats = season_stats.mean()
season_stats = season_stats.reset_index()
season_stats = season_stats.sort_values(["Season", "TeamID"])
season_stats = season_stats.reset_index(drop=True)
print(f"season_stats   {season_stats.shape[0]:,}")
p(season_stats)
print()

game_stats_1_2     166,128


,Season,DayNum,TeamID_1,TeamID_2,Part,Score_1,Score_2,FGM_1,FGM_2,FGA_1,FGA_2,FGM3_1,FGM3_2,FGA3_1,FGA3_2,FTM_1,FTM_2,FTA_1,FTA_2,OR_1,OR_2,DR_1,DR_2,Ast_1,Ast_2,TO_1,TO_2,Stl_1,Stl_2,Blk_1,Blk_2,PF_1,PF_2,Spread_1,Spread_2
0,2010,7,1107,1393,RegularSeason,43,75,15,29,55,60,5,2,28,17,8,15,14,31,12,14,27,32,10,21,32,21,12,21,4,6,27,16,-32,32
1,2010,7,1108,1326,RegularSeason,60,100,21,39,61,73,7,14,17,33,11,8,20,12,11,13,24,34,7,29,13,6,4,8,5,6,16,19,-40,40
2,2010,7,1143,1293,RegularSeason,75,70,24,26,52,52,5,8,12,21,22,10,32,15,13,11,19,20,15,11,12,17,8,7,1,3,14,24,5,-5
3,2010,7,1198,1314,RegularSeason,72,88,25,34,68,61,8,4,23,13,14,16,17,19,13,12,16,32,15,23,20,26,14,10,2,8,18,19,-16,16
4,2010,9,1143,1178,RegularSeason,95,61,29,20,61,57,7,4,17,18,30,17,35,27,15,14,30,18,11,14,10,15,5,4,3,2,20,21,34,-34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166123,2025,106,3150,3378,RegularSeason,52,70,18,25,58,59,5,4,17,17,11,16,14,18,7,8,24,30,5,15,15,12,8,9,2,3,15,16,-18,18
166124,2025,106,3194,3349,RegularSeason,39,72,10,30,55,63,3,10,17,24,16,2,23,4,4,5,25,39,5,21,10,10,4,6,3,5,9,14,-33,33
166125,2025,106,3242,3416,RegularSeason,63,58,17,23,46,63,4,7,17,26,25,5,28,11,5,15,26,21,9,11,12,12,4,4,5,2,16,21,5,-5
166126,2025,106,3329,3428,RegularSeason,68,64,23,25,63,58,6,5,23,23,16,9,18,16,18,8,25,18,9,16,17,11,6,8,3,6,21,20,4,-4



Y                  119,167


,Season,TeamID_1,TeamID_2,Spread_1
0,2010,1102,1140,-30.5
1,2010,1102,1149,10.0
2,2010,1102,1161,-13.0
3,2010,1102,1283,-10.0
4,2010,1102,1294,-24.0
...,...,...,...,...
119162,2025,3474,3480,-7.0
119163,2025,3475,3478,29.0
119164,2025,3476,3478,-2.0
119165,2025,3476,3479,12.5



reg_season         164,300
game_stats_1_o     164,300
game_stats_2_o     164,300
game_stats_1_o+2_o 328,600

game_stats_o_d     328,600


,Season,DayNum,TeamID_o,TeamID_d,Score_o,Score_d,FGM_o,FGM_d,FGA_o,FGA_d,FGM3_o,FGM3_d,FGA3_o,FGA3_d,FTM_o,FTM_d,FTA_o,FTA_d,OR_o,OR_d,DR_o,DR_d,Ast_o,Ast_d,TO_o,TO_d,Stl_o,Stl_d,Blk_o,Blk_d,PF_o,PF_d,Spread_o,Spread_d
0,2010,7,1107,1393,43,75,15,29,55,60,5,2,28,17,8,15,14,31,12,14,27,32,10,21,32,21,12,21,4,6,27,16,-32,32
1,2010,7,1107,1393,43,75,15,29,55,60,5,2,28,17,8,15,14,31,12,14,27,32,10,21,32,21,12,21,4,6,27,16,-32,32
2,2010,7,1108,1326,60,100,21,39,61,73,7,14,17,33,11,8,20,12,11,13,24,34,7,29,13,6,4,8,5,6,16,19,-40,40
3,2010,7,1108,1326,60,100,21,39,61,73,7,14,17,33,11,8,20,12,11,13,24,34,7,29,13,6,4,8,5,6,16,19,-40,40
4,2010,7,1143,1293,75,70,24,26,52,52,5,8,12,21,22,10,32,15,13,11,19,20,15,11,12,17,8,7,1,3,14,24,5,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328595,2025,106,3242,3416,63,58,17,23,46,63,4,7,17,26,25,5,28,11,5,15,26,21,9,11,12,12,4,4,5,2,16,21,5,-5
328596,2025,106,3329,3428,68,64,23,25,63,58,6,5,23,23,16,9,18,16,18,8,25,18,9,16,17,11,6,8,3,6,21,20,4,-4
328597,2025,106,3329,3428,68,64,23,25,63,58,6,5,23,23,16,9,18,16,18,8,25,18,9,16,17,11,6,8,3,6,21,20,4,-4
328598,2025,106,3398,3404,52,73,20,24,60,57,3,5,19,14,9,20,15,28,12,13,22,24,8,15,20,19,12,10,2,6,20,13,-21,21



season_stats   10,722


,Season,TeamID,Score_o,FGM_o,FGA_o,FGM3_o,FGA3_o,FTM_o,FTA_o,OR_o,DR_o,Ast_o,TO_o,Stl_o,Blk_o,PF_o,Spread_o,TeamID_d,Score_d,FGM_d,FGA_d,FGM3_d,FGA3_d,FTM_d,FTA_d,OR_d,DR_d,Ast_d,TO_d,Stl_d,Blk_d,PF_d,Spread_d
0,2010,1102,55.620690,20.000000,45.586207,5.620690,18.103448,10.000000,15.689655,6.758621,19.862069,12.275862,12.793103,4.827586,1.517241,17.551724,-7.344828,1338.689655,62.965517,21.448276,47.103448,7.206897,20.827586,12.862069,18.827586,8.862069,22.206897,13.000000,11.655172,5.931034,2.931034,16.034483,7.344828
1,2010,1103,71.030303,25.090909,57.636364,6.606061,19.515152,14.242424,21.393939,13.454545,22.696970,13.515152,13.666667,6.242424,3.181818,19.878788,4.575758,1276.939394,66.454545,22.787879,53.787879,4.969697,15.272727,15.909091,22.545455,11.424242,22.242424,10.848485,14.333333,7.060606,2.606061,19.333333,-4.575758
2,2010,1104,68.500000,24.718750,56.062500,5.468750,15.625000,13.593750,19.218750,12.062500,23.125000,13.406250,12.812500,7.781250,3.968750,18.218750,3.718750,1271.750000,64.781250,23.000000,54.187500,5.656250,17.812500,13.125000,19.781250,12.000000,22.437500,11.187500,13.937500,5.812500,4.093750,16.656250,-3.718750
3,2010,1105,63.826087,21.173913,57.173913,3.826087,13.565217,17.652174,27.521739,13.956522,21.608696,9.695652,15.913043,9.391304,5.217391,20.652174,-6.478261,1243.521739,70.304348,24.826087,56.913043,5.130435,16.347826,15.521739,24.521739,15.260870,27.173913,14.304348,18.826087,8.086957,5.304348,20.826087,6.478261
4,2010,1106,62.960000,20.720000,53.520000,6.160000,19.720000,15.360000,23.800000,12.840000,21.880000,12.520000,15.440000,7.520000,4.120000,20.720000,-3.840000,1243.080000,66.800000,22.600000,50.280000,5.200000,14.240000,16.400000,24.400000,10.080000,24.480000,11.720000,16.040000,7.520000,3.480000,20.880000,3.840000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10717,2025,3473,70.500000,23.500000,51.000000,8.500000,20.000000,15.000000,19.000000,8.500000,21.500000,15.000000,18.500000,10.000000,3.000000,16.000000,7.500000,3475.000000,63.000000,22.500000,58.000000,6.500000,20.500000,11.500000,14.000000,9.000000,15.500000,11.500000,15.000000,9.500000,2.500000,19.000000,-7.500000
10718,2025,3474,49.000000,15.000000,53.000000,5.000000,13.000000,14.000000,22.000000,11.000000,15.000000,8.000000,20.000000,6.000000,0.000000,23.000000,-7.000000,3480.000000,56.000000,18.000000,42.000000,7.000000,20.000000,13.000000,26.000000,7.000000,21.000000,8.000000,22.000000,10.000000,2.000000,18.000000,7.000000
10719,2025,3475,71.000000,31.000000,62.000000,8.000000,18.000000,1.000000,5.000000,8.000000,26.000000,22.000000,11.000000,9.000000,0.000000,15.000000,29.000000,3478.000000,42.000000,14.000000,52.000000,7.000000,35.000000,7.000000,10.000000,9.000000,18.000000,9.000000,12.000000,3.000000,3.000000,8.000000,-29.000000
10720,2025,3476,69.333333,25.666667,57.000000,8.333333,22.000000,9.666667,12.000000,8.333333,25.333333,16.333333,17.666667,7.000000,1.666667,18.666667,7.666667,3478.666667,61.666667,23.000000,59.000000,5.333333,23.666667,10.333333,13.333333,7.333333,21.333333,9.666667,16.666667,7.666667,1.000000,15.333333,-7.666667
